In [6]:
### Notebook used to test & compare outputs between MATLAB & Python implementations
## Can run entire notebook at once, some cells expected to be N/A or False for certain methods, see in-line comments, will output txt of results

In [7]:
import sys
from scipy import fft
from scipy.fft import fftshift
import numpy as np
from multiprocessing import Pool
from scipy.stats import pearsonr
import Bio
from functools import partial

import pywt
# import os

from statistics import median, mean
# from one_dimensional_num_mapping import *

# plotting

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# from classification import classify_dismat
# from helpers import *
# from visualisation import dimReduction
# set up
# if os.path.exists("Sequence_database.idx"):
#     os.remove("Sequence_database.idx")


In [8]:
#All validation tests run with Primates datasets from https://doi.org/10.1186/s12864-019-5571-y  

data_set = '/Users/dolteanu/local_documents/Coding/MLDSP_dev_git/data/Primates/fastas'
metadata = '/Users/dolteanu/local_documents/Coding/MLDSP_dev_git/data/Primates/metadata.csv'


In [9]:
#Function found in preprocessing.py
import os
import random
import pandas as pd
from Bio import SeqIO
from collections import Counter

def preprocessing(data_set, max_clust_size, metadata):
    """Preprocessing of fasta sequences using BioPython into a database of
    SeqRecord objects each representing a unique sequence, can handle multiple
    sequence fastas.

    seqs: main sequence database, dictionary-like object, no info on clusters
    cluster_names: list of all cluster names from sub-directory names.

    number_of_clusters: integer of the total count of clusters.

    cluster_sample_info: Dictionary with keys=cluster_names and values =
    a tuple consisting of: (number of samples in cluster,
        a list of accession ids corresponding to sequences of that cluster).

    total_seq: integer of the total sequence count in dataset.

    cluster_dict: depracated with cluster_sample_info, will be removed
    """
    # Dictionary to store SeqIO
    seq_dict = {}
    # dictionary with Accession ID as keys and cluster name as values
    cluster_dict = {}
    bad_keys=[]
   
    # Iterate through all fasta files
    for f in sorted(os.listdir(data_set)):
        file = os.path.join(data_set, f)
        with open(file) as handle:
            # SeqIO.index_db() is read only & can't multiprocess, SeqIO.index doesnt take file handle for multi-file, SeqIO_to_dict is all in memory, SeqIO.parse you only get to iterate over data once
    #         # single dict
            seq_dict.update(SeqIO.to_dict(SeqIO.parse(handle, "fasta")))
            
    cluster_dict = pd.read_csv(metadata,header=None, index_col=0, sep=None, engine='python').squeeze("columns").to_dict()
    cluster_stats = Counter(cluster_dict.values())
    
    # for key in seq_dict.keys():
    #     if not key in cluster_dict:
    #         bad_keys.append(key)
    
    # print(bad_keys)
    #Might affect order of labels & lead to mislabelling
    # for item in bad_keys:
    #     seq_dict.pop(item)
    
    total_seq = len(seq_dict)
    return seq_dict, total_seq, cluster_dict, cluster_stats


def old_preprocessing(data_set, max_clust_size):
    # Dictionary to store SeqIO
    seq_dict = {}
    cluster_names = sorted(os.listdir(data_set))
    # dictionary with Accession ID as keys and cluster name as values
    cluster_dict = {}
    # number of samples in each cluster
    # cluster_samples_info = {}
    # count of the number of clusters as int
    cluster_stats={}
    # Iterate over each cluster (top level directories)
    for cluster in cluster_names:
        files = os.listdir(os.path.join(data_set, cluster))
        files = [os.path.join(data_set, cluster, f) for f in files]
        # paths.extend(files)
        # get path for cluster as str
        cluster_path = os.path.join(data_set, cluster)
        # get names of files in cluster as list of str
        file_name = sorted(os.listdir(cluster_path))
        cluster_stats.update({cluster:len(file_name)})
        temp_dict={}
        # Iterate over each file in the cluster
        for file in file_name:
            # get path for each file in cluster as str
            file_path = os.path.join(cluster_path, file)
            # Required to use SeqIO.index to generate dictionary of SeqRecords (parsed on demand in main script)

            # SeqIO.index doesnt take file handle to index multiple seqs to a
            # single dict

            # Not sure if storing the dict like object of SeqIO.index in a dict forces loading into memory (performance penalty)
            seqs = SeqIO.index(file_path, "fasta"
            #,key_function=get_accession
            )
            seq_dict.update(seqs)
            # Generate second dictionary for cluster info
            for accession_id in seqs.keys():
                cluster_dict.update({accession_id: cluster})
        # if len(temp_dict) >= max_clust_size:
        #     subset = dict(random.sample(temp_dict.items(), max_clust_size))
        #     print(len(subset))
        #     seq_dict.update(subset)
        # else:
        #     seq_dict.update(temp_dict)
        # for accession_id in seq_dict.keys():
        #     cluster_dict.update({accession_id: cluster}) 
    total_seq = len(seq_dict)
    del temp_dict 
    del seqs
    return seq_dict, total_seq, cluster_dict, cluster_stats

In [10]:
#Functions found in one_dimensional_num_mapping.py
import numpy as np

def num_mapping_AT_CG(sq):
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t == 'A':
            numSeq[k] = 1
        elif t == 'C':
            numSeq[k] = -1
        elif t == 'G':
            numSeq[k] = -1
        elif t == 'T':
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_justA(sq):
    a = "A"
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == a:
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_justC(sq):
    c = "C"
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == c:
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_justG(sq):
    g = "G"
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == g:
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_justT(sq):
    t_ = "T"
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == t_:
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_Real(sq):
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == "A":
            numSeq[k] = -1.5
        elif t.upper() == "C":
            numSeq[k] = 0.5
        elif t.upper() == "G":
            numSeq[k] = -0.5
        elif t.upper() == "T":
            numSeq[k] = 1.5
        else:
            pass
    return numSeq


def num_mapping_PP(sq):
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == "A":
            numSeq[k] = -1
        elif t.upper() == "C":
            numSeq[k] = 1
        elif t.upper() == "G":
            numSeq[k] = -1
        elif t.upper() == "T":
            numSeq[k] = 1
        else:
            pass
    return numSeq


def num_mapping_IntN(sq):
    dob = ['T', 'C', 'A', 'G']
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        tp = dob.index(t) + 1
        numSeq[k] = tp
    return numSeq


def num_mapping_Int(sq):
    dob = ['T', 'C', 'A', 'G']
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        tp = dob.index(t)
        numSeq[k] = tp
    return numSeq


def num_mapping_EIIP(sq):
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == "A":
            numSeq[k] = 0.1260
        elif t.upper() == "C":
            numSeq[k] = 0.1340
        elif t.upper() == "G":
            numSeq[k] = 0.0806
        elif t.upper() == "T":
            numSeq[k] = 0.1335
        else:
            pass
    return numSeq


def num_mapping_Atomic(sq):
    length = len(sq)
    numSeq = np.zeros(length)
    for k in range(0, length):
        t = sq[k]
        if t.upper() == "A":
            numSeq[k] = 70
        elif t.upper() == "C":
            numSeq[k] = 58
        elif t.upper() == "G":
            numSeq[k] = 78
        elif t.upper() == "T":
            numSeq[k] = 66
        else:
            pass
    return numSeq


def num_mapping_Codons(sq):
    # Authored by Wanxin Li @wxli0
    length = len(sq)
    numSeq = np.zeros(length)
    codons = ['TTT','TTC','TTA','TTG','CTT','CTC','CTA','CTG','TCT','TCC','TCA','TCG','AGT','AGC','TAT','TAC',
              'TAA','TAG','TGA','TGT','TGC','TGG','CCT','CCC','CCA','CCG','CAT','CAC','CAA','CAG','CGT','CGC',
              'CGA','CGG','AGA','AGG','ATT','ATC','ATA','ATG','ACT','ACC','ACA','ACG','AAT','AAC','AAA','AAG',
              'GTT','GTC','GTA','GTG','GCT','GCC','GCA','GCG','GAT','GAC','GAA','GAG','GGT','GGC','GGA','GGG']
    # for k in range(0, length):
    #     if k < length-1:
    #         t = sq[k:k+3]
    #     elif k == length-1:
    #         t = sq[k:k+2] + sq[0]
    #     else:
    #         t = sq[k] + sq[0:2]
    #     tp = codons.index(t)
    #     numSeq[k] = tp
    for idx in range(length):
        if idx <= (length-3):
            t = sq[idx:idx+3]
        elif idx == (length-2):
            t = sq[idx:idx+2]+sq[0:1]
        else:
            t = sq[idx]+sq[0:2]
        tp = codons.index(t)
        numSeq[idx] = tp
    return numSeq


def num_mapping_Doublet(sq):
    # Authored by Wanxin Li @wxli0
    """computes Doublet representation
    Keyword arguments:
    sq: sequence
    """
    sq_len = len(sq)
    doublet = ['AA', 'AT', 'TA', 'AG', 'TT', 'TG', 'AC',
               'TC', 'GA', 'CA', 'GT', 'GG', 'CT', 'GC', 'CG', 'CC']
    numSeq = np.zeros(len(sq))
    # alpha = 0 # TODO: remove alpha for now, if alpha is added, then Codons also needs to be updated

    for idx in range(sq_len):
        # if alpha == 0:
        if idx < (sq_len-1):
            t = sq[idx:idx+2]
        else:
            t = sq[idx]+sq[0]
        tp = doublet.index(t)
        numSeq[idx] = tp
    return numSeq


In [11]:
#Functions found in helpers.py
import math, sys
import numpy as np
from statistics import median, mean
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

def length_calc(seq_list):
    """calculates length stats

    Keyword arguments:
    seq_list: a list of squence
    """
    len_list = map(len, seq_list)
    max_len = max(len_list)
    min_len = min(len_list)
    mean_len = mean(len_list)
    med_len = median(len_list)

    return max_len, min_len, mean_len, med_len

def inter_cluster_dist(clsuter,unique_clusters,distance_matrix, cluster_num):
    avg_dist = np.zeros((cluster_num,cluster_num))
    c_ind = np.zeros(cluster_num)
    for h in range(cluster_num):
        c_ind[h] = (clsuter == unique_clusters[h])
    
    for i in range(cluster_num):
        for j in range(i+1, cluster_num):
            if i==j:         
                continue           
            else:
                dT = distance_matrix[c_ind[i],c_ind[j]]
                avg_dist[i,j] = np.mean(np.transpose(dT), 1)  
                avg_dist[j,i] = avg_dist[i,j]
    return avg_dist


In [15]:
#Function from cgr.py
import numpy as np

def cgr(chars, order, k):
    """computes CGR representation in standard format: C top-left, G top-right, A bottom-left, T bottom-right

    Keyword arguments:
    chars: sequence
    order: chars to include in CGR
    k: value of k-mer
    """
    # set a numpy array of size 2^k,2^k  # remember that arrays are numbered top to bottom & left to right, unlike coordinate plots which go bottom up and left to right
    out = np.zeros((2**k,2**k))
    # set starting point of cgr plotting in the middle of cgr (x,y)
    x = 2**(k-1) 
    y = 2**(k-1)

    for i in range(len(chars)):
        char = chars[i]
        # devide x coordiate in half, moving it halfway to the left, this is correct if base is C or A
        x = int(x/2)
        # check to see if base is actually a G or T
        if char == order[2] or char == order[3]:  # if the nucleotide is G or T
            # add 2^(k-1) aka half the cgr length to the x value, brining it from 1/4 to 3/4
            x += 2**(k-1)
        # devide y coordiate in half, moving it halfway to the top, this is correct if base is C or G
        y = int(y/2)
        if char == order[0] or char == order[3]:  # if the nucleotide is A or T
            # add 2^(k-1) aka half the cgr length to the y value, brining it from 1/4 to 3/4
            y += 2**(k-1)
        # if i+1 is greater than or equal to k (i.e. if the position of the base is greater than k )
        if (i+1) >= k:
            # add plus 1 to the positions y & x in the cgr array
            out[y][x] += 1
    return out


In [12]:
#Functions found in visualisation.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import MDS, TSNE
from skbio.stats.ordination import pcoa

def dimReduction(data, n_dim, method):
    """
    Function will take in a nxm 2d-array and reduce the dimensions of the data using a specified dimensionality
    reduction technique (PCA, MDS, or TSNE).
    :param np.array data: input data to be transformed
    :param int n_dim: dimensions to reduce to
    :param str method: which method to use (either 'pca', 'mds', or 'tsne')
    :return np.array transformed: nxn_dim array of tranformed data
    """
    if method == 'pca':
        pca = PCA(n_components=n_dim,svd_solver='full')
        transformed = pca.fit_transform(data)
        return transformed
    elif method == 'mds':
        mds = pcoa(data, number_of_dimensions=n_dim)
        transformed = mds.samples
        return transformed
    elif method == 'tsne':
        tsne = TSNE(n_components=n_dim)
        transformed = tsne.fit_transform(data)
        return transformed
        

In [13]:
#Functions from classification.py
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict

def classify_dismat(dismat, alabels, folds, total, saveModels=False):

    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=23)
    model_names = {'LinearDiscriminant':LinearDiscriminantAnalysis() #matlab doesn't specify what solver it uses, orginal code used (shrinkage) gamma=0 
                   ,'LinearSVM':SVC(kernel='linear',cache_size=1000,decision_function_shape='ovo'), 'QuadSVM':SVC(kernel='poly', degree=2,cache_size=1000,decision_function_shape='ovo'),
                   'KNN':KNeighborsClassifier(n_neighbors=1, leaf_size=50, metric='euclidean', weights='uniform', algorithm='brute')
                   }
    # use 2 additional classifiers if <= 2000 sequences
    # if total <= 2000:
    #     model_names['SubspaceDiscriminant'] = SVC(kernel='rbf')
    #     model_names['SubspaceKNN'] = None

    accuracies = defaultdict(list) # dictionary with key: modelname, value: list containing accuracies
    confMatrixDict = defaultdict(list) # dictionary with key: modelname, value: list containing confusion matrix displays
    misclassifiedIdx = defaultdict(list) # dictionary with key: modelname, value: list containing indices/sequences of dismat that have been misclassifed

    # Loop through each model
    for modelName in model_names:
        model = model_names.get(modelName)
        print(model)
        # Create pipeline model
        if modelName in ['LinearSVM', 'QuadSVM', 'KNN']:
            pipeModel = make_pipeline(StandardScaler(), model)
        else:
            pipeModel = make_pipeline(model)
            
        for train_index, test_index in kf.split(dismat, alabels):
            X_train = dismat[train_index]
            X_test = dismat[test_index]
            y_train = [alabels[i] for i in train_index]
            y_test = [alabels[i] for i in test_index]

            # Fit the pipeline model
            pipeModel.fit(X_train, y_train)
            prediction = pipeModel.predict(X_test)
            # Compute and store accuracy of model
            accuracies[modelName].append(accuracy_score(y_test, prediction))
            print(accuracy_score(y_test, prediction))
            # Generate and store confusion matrix
            # cm = confusion_matrix(y_test, prediction, labels=list(np.unique(alabels)), normalize=None)
            # confMatrixDict[modelName].append(cm)

            # Store indices (of dismat) of misclassified sequences
            for i in range(len(prediction)):
                # if prediction incorrect, add to list of misclassified indices for the model
                if prediction[i] != y_test[i]:
                    misclassifiedIdx[modelName].append(test_index[i])

    # For each model, Calculate mean of accuracies across 10 folds & Sum all confusion matrices across 10 folds
    meanModelAccuracies = {} # key: modelName, value: mean accuracy value for model
    aggregatedCMatrix = {} # key: modelName, value: summed Confusion Matrix for model
    for modelName in accuracies:
        meanModelAccuracies[modelName] = np.mean(accuracies.get(modelName))
        aggregatedCMatrix[modelName] = np.sum(confMatrixDict.get(modelName), axis=0)

    # Mean accuracy value across all classifiers
    avgAccuracy = sum(meanModelAccuracies.values()) / len(meanModelAccuracies)

    return avgAccuracy, meanModelAccuracies, aggregatedCMatrix, dict(misclassifiedIdx)

# Plots and returns a ConfusionMatrix Display object from a raw array
def displayConfusionMatrix(confMatrix, alabels):
    # generate cm image and plot
    confMatrixDisplayObj = ConfusionMatrixDisplay(confusion_matrix=confMatrix, display_labels=list(np.unique(alabels)))
    confMatrixDisplayObj.plot(cmap='Blues', colorbar= False)

    # access raw cm array: cm_disp.confusion_matrix
    # alternative to display: cm_disp = plot_confusion_matrix(pipeModel, X_test, y_test, normalize=None, cmap='Blues', colorbar= False)

    return confMatrixDisplayObj


In [16]:
test_set = 'NoData'
seq_to_test = 0
min_seq_len = 0
max_clust_size = 5000000
frags_per_seq = 1

methods_list = {0: cgr, 1: num_mapping_PP, 2: num_mapping_Int, 3: num_mapping_IntN, 5: num_mapping_Doublet, 6: num_mapping_Codons, 7: num_mapping_Atomic,
                8: num_mapping_EIIP, 9: num_mapping_AT_CG, 10: num_mapping_justA, 11: num_mapping_justC, 12: num_mapping_justG, 13: num_mapping_justT, 14: 'PuPyCGR', 15: '1DPuPyCGR'}


In [17]:
def compute_cgr_PuPyCGR(seq_index):
    # seqs is the sequence database, it is being called by accession number
    # (keys) which is being iterated over all seq_index,
    # (count of all sequences in uploaded dataset) .seq calls the string
    seq = str(seq_dict[keys[seq_index]].seq)
    seq_new = seq
    if method_num == 14:
        seq_new = seq_new.replace('G', 'A')
        seq_new = seq_new.replace('C', 'T')
    cgr_output = cgr(seq_new, 'ACGT', k_val)  # shape:[2^k, 2^k]
    # shape:[2^k, 2^k] # may not be appropriate to take by column
    # axis=0 takes fft by column, consistent with MATLAB 
    fft_output = fft.fft(cgr_output,axis=0)
    # order='F' flattens the array in fortran index consistent with MATLAB, shape: [1,2^k*2^k], list makes sorting easy for comparison of distance matrices
    abs_fft_output = np.abs(fft_output.flatten(order='F'))
    return abs_fft_output, fft_output, cgr_output, seq_new, seq  # flatted into 1d array


In [18]:
def compute_1DPuPyCGR(seq_index):
    seq = str(seq_dict[keys[seq_index]].seq)
    # creates PuPyCGR
    seq_new = seq.replace('G', 'A')
    seq_new = seq_new.replace('C', 'T')
    cgr_raw = cgr(seq_new, 'ACGT', k_val)
    # takes only the last (bottom) row but all columns of cgr to make 1DPuPyCGR
    cgr_output = cgr_raw[-1, :]
    # shape:[1, 2^k] # may not be appropriate to take by column
    fft_output = fft.fft(cgr_output)
    abs_fft_output = np.abs(fft_output.flatten(order='F'))
    return abs_fft_output, fft_output, cgr_output, seq_new, seq # flatted into 1d array


In [19]:
def one_dimensional_num_mapping_wrapper(seq_index):
    # normalize sequences to median sequence length of cluster
    seq = str(seq_dict[keys[seq_index]].seq)
    if len(seq) >= med_len:
        seq_new = seq[0:round(med_len)]
    else:
        seq_new=seq
    num_seq = method(seq_new)
    if len(num_seq) < med_len:
        pad_width = int(med_len - len(num_seq))
        num_seq = pywt.pad(num_seq, pad_width, 'antisymmetric')[pad_width:]
    fft_output = fft.fft(num_seq)
    abs_fft_output = np.abs(fft_output.flatten()) #order='F' for testing abs_fft_outputs vs matlab
    return abs_fft_output, fft_output, num_seq, seq


In [ ]:
##Deprecated, not currently used, may be required for comparing upgma trees or mds
# def compute_pearson_coeffient(x, y):
#     r = pearsonr(x, y)[0]
#     normalized_r = (1-r)/2
#     return normalized_r

# def compute_pearson_coeffient_wrapper(i, j, abs_fft_output_list):
#     # print(abs_fft_output_list)
#     # x = abs_fft_output_list[i]
#     # y = abs_fft_output_list[indices[1]]
#     # print(x)
#     # print(y)
#     return compute_pearson_coeffient(abs_fft_output_list[i], abs_fft_output_list[j])


In [ ]:
# Start of validation, iterating over all available methods, 
from IPython.display import clear_output
for f in methods_list:
    method_num = f
    print(method_num)
    method = methods_list[f]
    if method_num in range(0,14):
        method_name = method.__name__
    else:
        method_name= method
    print(method_name)
    file = open(f"./dump/{method_name}_validation.txt", "x")
    # all matlab test .mat files were run at k=4
    k_val = 4  # used only for CGR-based representations(if methodNum=1,15,16)
    #seq_dict, total_seq, cluster_dict, cluster_stats = old_preprocessing(data_set,max_clust_size) #uncomment if using nested folder structure data
    # comment line below if using nested folder data instead of metadata.csv
    seq_dict, total_seq, cluster_dict, cluster_stats = preprocessing(data_set,max_clust_size, metadata)
    keys = list(seq_dict.keys())
    values = list(cluster_dict.values())
    # Could be parallelized in the future
    seqs_length = [len(seq_dict[keys[i]].seq) for i in range(total_seq)]
    med_len = median(seqs_length)
    labels = [cluster_dict[x] for x in seq_dict.keys()]
    fft_output_list = []
    abs_fft_output_list = []
    cgr_output_list = []
    seq_new_list = []
    seq_list=[]
    print('Generating numerical sequences, applying DFT, computing magnitude spectra .... \n')
    for seq_index in range(total_seq):
        if method_num == 0 or method_num == 14:
            abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_cgr_PuPyCGR(seq_index)
            abs_fft_output_list.append(abs_fft_output)
            fft_output_list.append(fft_output)
            cgr_output_list.append(cgr_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)
            
        elif method_num == 15:
            abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_1DPuPyCGR(seq_index)
            abs_fft_output_list.append(abs_fft_output)
            fft_output_list.append(fft_output)
            cgr_output_list.append(cgr_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)

        else:
            abs_fft_output, fft_output, seq_new, seq = one_dimensional_num_mapping_wrapper(seq_index)
            fft_output_list.append(fft_output)
            abs_fft_output_list.append(abs_fft_output)
            seq_new_list.append(seq_new)
            seq_list.append(seq)
    
    print('Building distance matrix')
    distance_matrix = (1-np.corrcoef(abs_fft_output_list))/2


    print('Performing classification .... \n')
    folds = 10
    if (total_seq < folds):
        folds = total_seq
    mean_accuracy, accuracies, confusion_matrix, misclassified_id = classify_dismat(distance_matrix, labels, folds, total_seq)
    print('Classification accuracy 5 classifiers:\n', accuracies, file=file)
    print('**** Processing completed ****\n')


    ### VALIDATION START ###
    from scipy.io import loadmat
    #load .mat file from your machine
    
    matlab = loadmat(f'./MATLAB data/Primates_testing_all_numerical_methods/primates_{method_name}.mat',simplify_cells=True)
    locals().update(matlab)
    # compare raw input sequences
    print(f"\nComparing raw input sequences:\n{seq_list==Seq}", file=file)

    # compare output sequences for cgr methods; 
    if method_num in [0,14,15]:
        print(f"\nCompare cgr inputs, only different from raw input sequences for non-standard cgr i.e Purine-Pyrimidine & 1D CGR:\n {seq_new_list==sequence}", file=file)
        print("\nComparing each sample's cgr output", file=file)
        for thing in range(total_seq):
            print(True==np.allclose(cgr_output_list[thing],nmValSH[thing]),file=file)
    else:
        print("\nCompare output of numerical representation:",file=file)
        for thing in range(total_seq):
            print(seq_new_list[thing]==nmValSH[thing],file=file)
    
    print("\nCompare fft outputs:",file=file)
    for thing in range(total_seq):
        print(np.allclose(fft_output_list[thing],f[thing]),file=file)
    
    print("\nCompare magnitudes of ffts:",file=file)
    for thing in range(total_seq):
        print(np.allclose(abs_fft_output_list[thing],lg[thing]),file=file)
    
    print(f"\nCompare distance matrix:\n{np.allclose(distance_matrix,disMat,atol=1e-15,rtol=1e-11)}",file=file)
    file.close()
    clear_output()
    # objects = dir()
    # for obj in objects:
    #     if not obj.startswith("__"):
    #         del globals()[obj]

In [9]:
seq_dict, total_seq, cluster_dict, cluster_stats = preprocessing(data_set,max_clust_size, metadata)


[]


/Users/dolteanu/.pyenv/versions/MLDSP/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [10]:
method_num = 0
k_val = 4

In [11]:
# variable holding all the keys (accession numbers) for corresponding clusters

keys = list(seq_dict.keys())

values = list(cluster_dict.values())
# Could be parallelized in the future

seqs_length = [len(seq_dict[keys[i]].seq) for i in range(total_seq)]
med_len = median(seqs_length)
labels = [cluster_dict[x] for x in seq_dict.keys()]
fft_output_list = []
abs_fft_output_list = []
cgr_output_list = []
seq_new_list = []
seq_list=[]


In [ ]:
# #Phylogenetic trees not compared
# def phylogenetic_tree(distance_matrix):
#     names = keys
#     matrix = triangle_matrix
#     distance_matrix = DistanceMatrix(names, matrix)
#     constructor = DistanceTreeConstructor()
#     nj_tree = constructor.nj(distance_matrix)
#     # newick may not be need to be quoted
#     neighbour_joining_tree = nj_tree.format('newick')
#     upgma = constructor.upgma(distance_matrix)
#     upgma_tree = upgma.format('newick')
#     print(upgma_tree, file=tree_print)
#     # can add code here for visualization with matplotlib


In [12]:
print('Generating numerical sequences, applying DFT, computing magnitude spectra .... \n')
for seq_index in range(total_seq):
    if method_num == 0 or method_num == 14:
        abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_cgr_PuPyCGR(seq_index)
        abs_fft_output_list.append(abs_fft_output)
        fft_output_list.append(fft_output)
        cgr_output_list.append(cgr_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)
        
    elif method_num == 15:
        abs_fft_output, fft_output, cgr_output, seq_new, seq = compute_1DPuPyCGR(seq_index)
        abs_fft_output_list.append(abs_fft_output)
        fft_output_list.append(fft_output)
        cgr_output_list.append(cgr_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)

    else:
        abs_fft_output, fft_output, seq_new, seq = one_dimensional_num_mapping_wrapper(seq_index)
        fft_output_list.append(fft_output)
        abs_fft_output_list.append(abs_fft_output)
        seq_new_list.append(seq_new)
        seq_list.append(seq)

Generating numerical sequences, applying DFT, computing magnitude spectra .... 



In [18]:
print('Building distance matrix')
# distance_matrix = []
# for i in range(total_seq):
#     for j in range(total_seq):
#         distance_matrix.append(compute_pearson_coeffient_wrapper(i,j,abs_fft_output_list)) 
# distance_matrix = np.array(distance_matrix).reshape(total_seq, total_seq)
# np.savetxt('distmat.txt', distance_matrix)


Building distance matrix


In [13]:
new_dist_mat = (1-np.corrcoef(abs_fft_output_list))/2

In [19]:
np.allclose(distance_matrix,new_dist_mat,atol=1e-15,rtol=1e-11)

True

In [ ]:
    # print('Building Phylogenetic Trees')
    # matrix_list = distance_matrix.tolist()
    # triangle_matrix = []
    # # loop to create triangle matrix as nested list
    # for i in range(total_seq):
    #     row = []
    #     row = (matrix_list[i][0:i+1])
    #     triangle_matrix.append(row)
    # # change filename to unique ID
    # tree_print = open('upgma.tree', 'a')
    # phylogenetic_tree(triangle_matrix)
    # tree_print.close()

In [14]:
print('Performing classification .... \n')
folds = 10
if (total_seq < folds):
    folds = total_seq
mean_accuracy, accuracies, confusion_matrix, misclassified_id = classify_dismat(new_dist_mat, labels, folds, total_seq)
# accuracy,avg_accuracy, clNames, cMat
# accuracies = [accuracy, avg_accuracy];
print('Classification accuracy 5 classifiers\n', accuracies)
print('**** Processing completed ****\n')

Performing classification .... 

LinearDiscriminantAnalysis()
0.9333333333333333
14
0.9333333333333333
14
0.8666666666666667
14
0.9333333333333333
14
0.8
14
0.8666666666666667
14
0.9333333333333333
14
0.8666666666666667
14
0.9285714285714286
13
1.0
13
SVC(kernel='linear')
1.0
14
1.0
14
1.0
14
1.0
14
0.9333333333333333
14
1.0
14
0.9333333333333333
14
0.9333333333333333
14
0.9285714285714286
13
0.9285714285714286
13
SVC(degree=2, kernel='poly')
0.8666666666666667
14
0.8666666666666667
14
0.8666666666666667
14
0.6
14
0.8666666666666667
14
0.7333333333333333
14
0.8666666666666667
14
0.9333333333333333
14
0.8571428571428571
13
0.9285714285714286
13
KNeighborsClassifier(algorithm='brute', leaf_size=50, metric='euclidean',
                     n_neighbors=1)
1.0
14
1.0
14
1.0
14
0.8666666666666667
14
1.0
14
0.8
14
0.8666666666666667
14
0.8666666666666667
14
0.8571428571428571
13
1.0
13
Classification accuracy 5 classifiers
 {'LinearDiscriminant': 0.9061904761904762, 'LinearSVM': 0.96571428571

In [15]:
# Save entire matlab workspace as .mat 
# Use matlab mldsp version from github as variables have been added for testing; make sure k_val (where applicable) and methods match
from scipy.io import loadmat
#load .mat file from your machine
matlab = loadmat('/Users/dolteanu/local_documents/Coding/MLDSP_dev_git/Validation/primates_cgr.mat',simplify_cells=True)
locals().update(matlab)

/Users/dolteanu/.pyenv/versions/MLDSP/lib/python3.8/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [16]:
# Testing convention in this notebook: python variable on the left, matlab variable on the right
# compare accession number
keys==AcNmb

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
#compare raw input sequences
seq_list==Seq

In [ ]:
#Run only for cgr based methods
# compare output sequences for cgr methods; regular cgr will be the same as previous cell since raw sequence is used for cgr generation
seq_new_list==sequence

In [ ]:
# Run only for 1D numerical representation methods
# Compare output of numerical representation
for thing in range(total_seq):
    print(seq_new_list[thing]==nmValSH[thing])

In [ ]:
# Run only for cgr methods
# compare CGRs
for thing in range(total_seq):
    print(True==np.allclose(cgr_output_list[thing],nmValSH[thing]))
        

In [ ]:
# Compare fft outputs
for thing in range(total_seq):
    print(np.allclose(fft_output_list[thing],f[thing]))

In [20]:
# Compare magnitudes of ffts
for thing in range(total_seq):
    print(np.allclose(abs_fft_output_list[thing],lg[thing]))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [19]:
#Compare distance matrices
#modify exponent of 'rtol' to find similarity of dist mats, atol should be left untouched as it tests precision of zeros (diagonal)
np.allclose(new_dist_mat,disMat,atol=1e-15,rtol=1e-11)

True

In [18]:
#Compare MDS outputs-currently not matching due to sklearn using SMACOF algorithm while matlab uses SVD.
scaled_distance_matrix = dimReduction(new_dist_mat, n_dim=3, method='pca')
np.isclose(scaled_distance_matrix,Y)

array([[False, False, False],
       [False, False, False],
       [False,  True, False],
       [False, False,  True],
       [False,  True, False],
       [False, False,  True],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False,  True],
       [False, False,  True],
       [False,  True, False],
       [False, False,  True],
       [False, False,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False,  True],
       [False, False, False],
       [False,  True, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [Fa